# Content Based Filtering

<b>Recommendation systems</b> are a collection of algorithms used to recommend items to users based on information taken from the user. These systems have become ubiquitous, and can be commonly seen in online stores, <b>movies databases</b> and job finders. In this notebook, we will explore Content-based recommendation systems and implement a simple version of one using Python and the Pandas library.


# Dataset

You can download Dataset from this link: https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-ML0101EN-SkillsNetwork/labs/Module%205/data/moviedataset.zip

# Import 

In [1]:
#Dataframe manipulation library
import pandas as pd
#Math functions, we'll only need the sqrt function so let's import only that
from math import sqrt
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Load Data

In [2]:
#Storing the movie information into a pandas dataframe
movies_df = pd.read_csv('C:/Users/Asus/Downloads/movies.csv')
#Storing the user information into a pandas dataframe
ratings_df = pd.read_csv('C:/Users/Asus/Downloads/ratings.csv')
#Head is a function that gets the first N rows of a dataframe. N's default is 5.
movies_df.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [3]:
movies_df.shape

(34208, 3)

In movies_df Dataframe, 34208 rows and 3 columns

# Preprocessing

In [4]:
#Using regular expressions to find a year stored between parentheses
#We specify the parantheses so we don't conflict with movies that have years in their titles
movies_df['year'] = movies_df.title.str.extract('(\(\d\d\d\d\))',expand=False)
#Removing the parentheses
movies_df['year'] = movies_df.year.str.extract('(\d\d\d\d)',expand=False)
#Removing the years from the 'title' column
movies_df['title'] = movies_df.title.str.replace('(\(\d\d\d\d\))', '')
#Applying the strip function to get rid of any ending whitespace characters that may have appeared
movies_df['title'] = movies_df['title'].apply(lambda x: x.strip())
movies_df.head()

,movieId,title,genres,year
0,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,1995
1,2,Jumanji,Adventure|Children|Fantasy,1995
2,3,Grumpier Old Men,Comedy|Romance,1995
3,4,Waiting to Exhale,Comedy|Drama|Romance,1995
4,5,Father of the Bride Part II,Comedy,1995


In [5]:
#Every genre is separated by a | so we simply have to call the split function on |
movies_df['genres'] = movies_df.genres.str.split('|')
movies_df.head()

,movieId,title,genres,year
0,1,Toy Story,"[Adventure, Animation, Children, Comedy, Fantasy]",1995
1,2,Jumanji,"[Adventure, Children, Fantasy]",1995
2,3,Grumpier Old Men,"[Comedy, Romance]",1995
3,4,Waiting to Exhale,"[Comedy, Drama, Romance]",1995
4,5,Father of the Bride Part II,[Comedy],1995


In [6]:
#Copying the movie dataframe into a new one since we won't need to use the genre information in our first case.
moviesWithGenres_df = movies_df.copy()

#For every row in the dataframe, iterate through the list of genres and place a 1 into the corresponding column
for index, row in movies_df.iterrows():
    for genre in row['genres']:
        moviesWithGenres_df.at[index, genre] = 1
#Filling in the NaN values with 0 to show that a movie doesn't have that column's genre
moviesWithGenres_df = moviesWithGenres_df.fillna(0)
moviesWithGenres_df.head()

,movieId,title,genres,year,Adventure,Animation,Children,Comedy,Fantasy,Romance,...,Horror,Mystery,Sci-Fi,IMAX,Documentary,War,Musical,Western,Film-Noir,(no genres listed)
0,1,Toy Story,"[Adventure, Animation, Children, Comedy, Fantasy]",1995,1.0,1.0,1.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,Jumanji,"[Adventure, Children, Fantasy]",1995,1.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,Grumpier Old Men,"[Comedy, Romance]",1995,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,Waiting to Exhale,"[Comedy, Drama, Romance]",1995,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,Father of the Bride Part II,[Comedy],1995,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Next, let's look at the ratings dataframe.

In [7]:
ratings_df.head()

,userId,movieId,rating,timestamp
0,1,169,2.5,1204927694
1,1,2471,3.0,1204927438
2,1,48516,5.0,1204927435
3,2,2571,3.5,1436165433
4,2,109487,4.0,1436165496


In [8]:
# no need of timestamp column so drop it
ratings_df.drop('timestamp',axis=1,inplace=True)

# Genres Data

In [9]:
genre_df = moviesWithGenres_df.drop('title',axis=1).drop('genres',axis=1).drop('year',axis=1)
genre_df.set_index('movieId',inplace=True)
genre_df.head()

,Adventure,Animation,Children,Comedy,Fantasy,Romance,Drama,Action,Crime,Thriller,Horror,Mystery,Sci-Fi,IMAX,Documentary,War,Musical,Western,Film-Noir,(no genres listed)
movieId,,,,,,,,,,,,,,,,,,,,
1,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# User Selection

Select user with their userId

For e.g: I select user whose id is 3

In [10]:
userId = int(input('Enter UserId: '))

Enter UserId: 3


In [11]:
user = ratings_df[ratings_df['userId']==userId]
user.reset_index(inplace=True)
user

,index,userId,movieId,rating
0,7,3,356,4.0
1,8,3,2394,4.0
2,9,3,2431,5.0
3,10,3,2445,4.0


# Filtering Movies

In [12]:
# Filter Genre from Genre_df
userGenre = genre_df[(genre_df.index).isin(user['movieId'].tolist())]
userGenre=userGenre.reset_index(drop=True)
userGenre

,Adventure,Animation,Children,Comedy,Fantasy,Romance,Drama,Action,Crime,Thriller,Horror,Mystery,Sci-Fi,IMAX,Documentary,War,Musical,Western,Film-Noir,(no genres listed)
0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Now we're ready to start learning the input's preferences!

To do this, we're going to turn each genre into weights. We can do this by using the input's reviews and multiplying them into the input's genre table and then summing up the resulting table by column. This operation is actually a dot product between a matrix and a vector, so we can simply accomplish by calling the Pandas "dot" function.


In [14]:
#Dot produt to get weights
userProfile = userGenre.transpose().dot(user['rating'])
#The user profile
userProfile

Adventure              0.0
Animation              4.0
Children               0.0
Comedy                 9.0
Fantasy                0.0
Romance                4.0
Drama                 13.0
Action                 0.0
Crime                  0.0
Thriller               0.0
Horror                 0.0
Mystery                0.0
Sci-Fi                 0.0
IMAX                   0.0
Documentary            0.0
War                    4.0
Musical                4.0
Western                0.0
Film-Noir              0.0
(no genres listed)     0.0
dtype: float64

With the input's profile and the complete list of movies and their genres in hand, we're going to take the weighted average of every movie based on the input profile and recommend the top 10 movies that most satisfy it.


In [15]:
recommendation_df = genre_df.dot(userProfile)
recommendation_df.head()

movieId
1    13.0
2     0.0
3    13.0
4    26.0
5     9.0
dtype: float64

In [16]:
#Sort our recommendations in descending order
recommendation_df = recommendation_df.sort_values(ascending=False)
#Just a peek at the values
recommendation_df.head(10)

movieId
1907      34.0
39768     34.0
26093     34.0
54978     30.0
79920     30.0
108514    30.0
914       30.0
60225     30.0
3199      30.0
2324      30.0
dtype: float64

# Final Recommendation Table
Now here's the recommendation table!

In [18]:
#The final recommendation table
print('For userId : ',userId,' Recommendation is given below:' )
movies_df.loc[movies_df['movieId'].isin(recommendation_df.head(10).keys())]

For userId :  3  Recommendation is given below:


,movieId,title,genres,year
897,914,My Fair Lady,"[Comedy, Drama, Musical, Romance]",1964
1824,1907,Mulan,"[Adventure, Animation, Children, Comedy, Drama...",1998
2240,2324,Life Is Beautiful (La Vita è bella),"[Comedy, Drama, Romance, War]",1997
3113,3199,Pal Joey,"[Comedy, Drama, Musical, Romance]",1957
8605,26093,"Wonderful World of the Brothers Grimm, The","[Adventure, Animation, Children, Comedy, Drama...",1962
10549,39768,Life is a Miracle (Zivot je cudo),"[Comedy, Drama, Musical, Romance, War]",2004
12097,54978,"Good Night, The","[Comedy, Drama, Fantasy, Musical, Romance]",2007
12789,60225,Tell It to the Marines,"[Comedy, Drama, Romance, War]",1926
15753,79920,Blue Skies,"[Comedy, Drama, Musical, Romance]",1946
22766,108514,Yeh Jawaani Hai Deewani,"[Comedy, Drama, Musical, Romance]",2013
